In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
# !pip install huggingface_hub
# !pip install ipywidgets
from huggingface_hub import notebook_login
from huggingface_hub import login

Reference: https://huggingface.co/docs/transformers/tasks/summarization#load-billsum-dataset

In [ ]:
news_dataset_cnndailymail = pd.read_csv( "/content/drive/MyDrive/cnn_dailymail/train.csv", nrows=80000)
news_dataset_cnndailymail.drop(columns=["id"], inplace=True )
news_dataset_cnndailymail.rename(columns={"highlights":"summary"},inplace=True)
# news_dataset_cnndailymail = news_dataset_cnndailymail[0:100]
rows_count = news_dataset_cnndailymail.shape[0]
news_dataset_cnndailymail

,article,summary
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...
79995,Tesco is at the centre of cruelty allegations ...,Customers can order the animals over the count...
79996,By . Daily Mail Reporter . PUBLISHED: . 10:42 ...,U.S-led peace talks falter after the deaths of...
79997,The chaotic scenes inside a New York subway st...,A passer-by captured cell phone footage of the...
79998,(GameTap.com) -- Everyone wants to be more phy...,Entire 'Wii Fit' setup doesn't take up much sp...


Converting Pandas Dataframe into hugging face dataset type


In [ ]:
# !pip install pyarrow
!pip install datasets
# !pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

### convert to Huggingface dataset
news_dataset_cnndailymail = Dataset(pa.Table.from_pandas(news_dataset_cnndailymail))
news_dataset_cnndailymail = news_dataset_cnndailymail.train_test_split(test_size=0.1)
news_dataset_cnndailymail

DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 72000
    })
    test: Dataset({
        features: ['article', 'summary'],
        num_rows: 8000
    })
})

Tokenization

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')

prefix = "summarize: "

def preprocess_function( data ):
    inputs = [prefix + doc for doc in data["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=data["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_news_dataset_cnndailymail = news_dataset_cnndailymail.map(preprocess_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/72000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Dynamic Padding

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small", return_tensors="tf")

Metric

In [ ]:
# !pip install -U git+https://github.com/huggingface/accelerate.git
from transformers import pipeline

# !pip install evaluate
# !pip install rouge_score
import evaluate
rouge = evaluate.load("rouge")

In [ ]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

Model Setup

In [ ]:
# import tensorflow
# AUTO = tensorflow.data.experimental.AUTOTUNE

# # # Create strategy from tpu
# tpu = tensorflow.distribute.cluster_resolver.TPUClusterResolver()
# tensorflow.config.experimental_connect_to_cluster(tpu)
# tensorflow.tpu.experimental.initialize_tpu_system(tpu)
# strategy = tensorflow.distribute.experimental.TPUStrategy(tpu)

# BATCH_SIZE = 16 * strategy.num_replicas_in_sync

In [ ]:
from transformers import create_optimizer, AdamWeightDecay
from transformers import TFAutoModelForSeq2SeqLM
import tensorflow as tf
import tensorflow
from transformers.keras_callbacks import KerasMetricCallback
from transformers.keras_callbacks import PushToHubCallback

In [ ]:
# with strategy.scope():
#     optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
#     model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")
#     # Dataset setup for training
#     tf_train_set = model.prepare_tf_dataset( tokenized_news_dataset_cnndailymail["train"], shuffle=True, batch_size=BATCH_SIZE, collate_fn=data_collator )
#     tf_test_set = model.prepare_tf_dataset( tokenized_news_dataset_cnndailymail["test"], shuffle=False, batch_size=BATCH_SIZE, collate_fn=data_collator )

#     model.compile(optimizer=optimizer)  # No loss argument!
# model.summary()

In [ ]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")
BATCH_SIZE = 8
# Dataset setup for training
tf_train_set = model.prepare_tf_dataset( tokenized_news_dataset_cnndailymail["train"], shuffle=True, batch_size=BATCH_SIZE, collate_fn=data_collator )
tf_test_set = model.prepare_tf_dataset( tokenized_news_dataset_cnndailymail["test"], shuffle=False, batch_size=BATCH_SIZE, collate_fn=data_collator )

model.compile(optimizer=optimizer)  # No loss argument!
model.summary()

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
       filepath="/content/drive/MyDrive/cnn_dailymail/finetune/best_model",
       monitor="val_loss",
       save_best_only=True,
       save_weights_only=False,
       mode="min",
       verbose=1,
       save_format="tf"  # Add this line
   )

In [ ]:
N_STEPS = int( (rows_count*0.1))//BATCH_SIZE
EPOCHS = 10
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_test_set)
lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0, mode='auto', min_delta=0.0001, cooldown=0,min_lr=0)
push_to_hub_callback = PushToHubCallback(output_dir="Fine_Tune_T5_Model_News_Summarization", tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/Fine_Tune_T5_Model_News_Summarization is already a clone of https://huggingface.co/AvianSpread/Fine_Tune_T5_Model_News_Summarization. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
from tensorflow.keras.utils import custom_object_scope
def register_custom_objects():
    custom_objects = {"AdamWeightDecay": AdamWeightDecay}
    return custom_objects
  optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, steps_per_epoch=N_STEPS, epochs=EPOCHS,callbacks=[lr_reduce,checkpoint_callback])

Epoch 1/10
1000/1000 [==============================] - ETA: 0s - loss: 1.8749
Epoch 1: val_loss improved from inf to 1.69848, saving model to /content/drive/MyDrive/cnn_dailymail/finetune/best_model
1000/1000 [==============================] - 1126s 1s/step - loss: 1.8749 - val_loss: 1.6985 - lr: 2.0000e-05
Epoch 2/10
1000/1000 [==============================] - ETA: 0s - loss: 1.8794
Epoch 2: val_loss improved from 1.69848 to 1.69405, saving model to /content/drive/MyDrive/cnn_dailymail/finetune/best_model
1000/1000 [==============================] - 1097s 1s/step - loss: 1.8794 - val_loss: 1.6940 - lr: 2.0000e-05
Epoch 3/10
1000/1000 [==============================] - ETA: 0s - loss: 1.8731
Epoch 3: val_loss improved from 1.69405 to 1.69368, saving model to /content/drive/MyDrive/cnn_dailymail/finetune/best_model
1000/1000 [==============================] - 1141s 1s/step - loss: 1.8731 - val_loss: 1.6937 - lr: 2.0000e-05
Epoch 4/10
1000/1000 [==============================] - ETA: 0

KeyboardInterrupt: 

###